In [ ]:
import numpy as np
import pandas as pd
import os

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
def align_yaxis(ax1, v1, ax2, v2):
    """adjust ax2 ylimit so that v2 in ax2 is aligned to v1 in ax1"""
    _, y1 = ax1.transData.transform((0, v1))
    _, y2 = ax2.transData.transform((0, v2))
    adjust_yaxis(ax2,(y1-y2)/2,v2)
    adjust_yaxis(ax1,(y2-y1)/2,v1)

def adjust_yaxis(ax,ydif,v):
    """shift axis ax by ydiff, maintaining point v at the same location"""
    inv = ax.transData.inverted()
    _, dy = inv.transform((0, 0)) - inv.transform((0, ydif))
    miny, maxy = ax.get_ylim()
    miny, maxy = miny - v, maxy - v
    if -miny>maxy or (-miny==maxy and dy > 0):
        nminy = miny
        nmaxy = miny*(maxy+dy)/(miny+dy)
    else:
        nmaxy = maxy
        nminy = maxy*(miny+dy)/(maxy+dy)
    ax.set_ylim(nminy+v, nmaxy+v)

In [ ]:
imu_file=os.path.expanduser('~/Documents/log/2019-01-14T17-48-13/logstream_ImuNoCov.bin')
odom_file=os.path.expanduser('~/Documents/log/2019-01-14T17-48-13/logstream_OdomNoCov.bin')
cmd_file=os.path.expanduser('~/Documents/log/2019-01-14T17-48-13/logstream_AccelCmd.bin')
esc_file=os.path.expanduser('~/Documents/log/2019-01-14T17-48-13/logstream_ActAdapEsc.bin')

In [ ]:
esc = np.dtype([('idx', '>u1'),
                ('cmdSec', '>u4'), ('cmdUsec', '>u4'), ('cmd', '>u4'), ('cmdVel', '>f8'),
                ('fbSec', '>u4'), ('fbUsec', '>u4'), ('angVel', '>f8'),
               
                ('length', '>u2'),
                ('counts', '>u2'),
                ('voltage', '>u2'),
                ('temperature', '>u2'),
                ('current', '>u2'),
                ('id', 'u1')])

imu = np.dtype([('recvSec', '>u4'), ('recvUsec', '>u4'),
               
                ('seq', '>u4'),
                ('timeBase', '>u2'), ('timeContext', '>u1'), ('timeSec', '>u4'), ('timeUsec', '>u4'),
                ('frame_id', '>u4'),
               
                ('qx', '>f8'), ('qy', '>f8'), ('qz', '>f8'), ('qw', '>f8'),

                ('wx', '>f8'), ('wy', '>f8'), ('wz', '>f8'),

                ('ax', '>f8'), ('ay', '>f8'), ('az', '>f8')])

odom = np.dtype([('recvSec', '>u4'), ('recvUsec', '>u4'),
               
                 ('seq', '>u4'),
                 ('timeBase', '>u2'), ('timeContext', '>u1'), ('timeSec', '>u4'), ('timeUsec', '>u4'),
                 ('frame_id', '>u4'),
                 
                 ('child_frame_id', '>u4'),

                 ('x', '>f8'), ('y', '>f8'), ('z', '>f8'),
               
                 ('qx', '>f8'), ('qy', '>f8'), ('qz', '>f8'), ('qw', '>f8'),

                 ('vx', '>f8'), ('vy', '>f8'), ('vz', '>f8'),

                 ('wx', '>f8'), ('wy', '>f8'), ('wz', '>f8')])

cmd = np.dtype([('recvSec', '>u4'), ('recvUsec', '>u4'),
               
                ('seq', '>u4'),
                ('timeBase', '>u2'), ('timeContext', '>u1'), ('timeSec', '>u4'), ('timeUsec', '>u4'),
                ('frame_id', '>u4'),

                ('ax', '>f8'), ('ay', '>f8'), ('az', '>f8'),

                ('alpha_x', '>f8'), ('alpha_y', '>f8'), ('alpha_z', '>f8')])

In [ ]:
imu_dat = np.fromfile(imu_file, dtype=imu)
imu_df = pd.DataFrame(imu_dat.byteswap().newbyteorder())

In [ ]:
odom_dat = np.fromfile(odom_file, dtype=odom)
odom_df = pd.DataFrame(odom_dat.byteswap().newbyteorder())

In [ ]:
cmd_dat = np.fromfile(cmd_file, dtype=cmd)
cmd_df = pd.DataFrame(cmd_dat.byteswap().newbyteorder())

In [ ]:
imu_time = imu_df.timeSec + imu_df.timeUsec / (1000.0 * 1000.0)
odom_time = odom_df.timeSec + odom_df.timeUsec / (1000.0 * 1000.0)
cmd_time = cmd_df.timeSec + cmd_df.timeUsec / (1000.0 * 1000.0)

In [ ]:
esc_dat = np.fromfile(esc_file, dtype=esc)
esc_df = pd.DataFrame(esc_dat.byteswap().newbyteorder())

mot0=esc_df.loc[(esc_df['idx'] == 0) & (esc_df['cmd'] <= 800)]
mot1=esc_df.loc[(esc_df['idx'] == 1) & (esc_df['cmd'] <= 800)]
mot2=esc_df.loc[(esc_df['idx'] == 2) & (esc_df['cmd'] <= 800)]
mot3=esc_df.loc[(esc_df['idx'] == 3) & (esc_df['cmd'] <= 800)]

mot0_cmdTime = mot0.cmdSec + mot0.cmdUsec / (1000.0 * 1000.0)
mot1_cmdTime = mot1.cmdSec + mot1.cmdUsec / (1000.0 * 1000.0)
mot2_cmdTime = mot2.cmdSec + mot2.cmdUsec / (1000.0 * 1000.0)
mot3_cmdTime = mot3.cmdSec + mot3.cmdUsec / (1000.0 * 1000.0)

mot0_fbTime = mot0.fbSec + mot0.fbUsec / (1000.0 * 1000.0)
mot1_fbTime = mot1.fbSec + mot1.fbUsec / (1000.0 * 1000.0)
mot2_fbTime = mot2.fbSec + mot2.fbUsec / (1000.0 * 1000.0)
mot3_fbTime = mot3.fbSec + mot3.fbUsec / (1000.0 * 1000.0)

In [ ]:
fig, ax1 = plt.subplots()

ax1.plot(mot0_fbTime, mot0.cmdVel, '')
ax1.plot(mot1_fbTime, mot1.cmdVel, '')
ax1.plot(mot2_fbTime, mot2.cmdVel, '')
ax1.plot(mot3_fbTime, mot3.cmdVel, '')

ax2 = ax1.twinx()
ax2.plot(cmd_time, cmd_df.alpha_z, 'r')
ax2.plot(odom_time, odom_df.wz, 'y')

align_yaxis(ax1, 0, ax2, 0)

ax1.set_xlim(938.5, 940.5)